# 03: Prompt Caching

In [ ]:
from anthropic import Anthropic

In [9]:
import os
if not os.environ["ANTHROPIC_API_KEY"]:
    print("environment variable ANTHROPIC_API_KEY not found, checking in api.txt")
    with open("api.txt") as i:
        api = i.read().strip()
        if not api:
            print("nothing found in api.txt, create a key in console.anthropic.com and paste it there")
        else:
            os.environ["ANTHROPIC_API_KEY"] = api
            print("key found in api.txt")  
else:
    print("environment variable ANTHROPIC_API_KEY found")    

environment variable ANTHROPIC_API_KEY not found, checking in api.txt
key found in api.txt


In [ ]:
client = Anthropic()
MODEL_NAME="claude-3-5-sonnet-20241022"

### Loading The Book

In [ ]:
with open('files/frankenstein.txt', 'r') as file:
    book_content = file.read()

In [ ]:
len(book_content)

In [ ]:
book_content[1000:2000]

## Uncached Request

In [ ]:
import time
def make_non_cached_api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content + "</book>"
                },
                {
                    "type": "text",
                    "text": "What happens in chapter 3?"
                }
            ]
        }
    ]

    start_time = time.time()
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=500,
        messages=messages,
    )
    end_time = time.time()

    return response, end_time - start_time

In [ ]:
non_cached_response, non_cached_time = make_non_cached_api_call()
print(f"Non-cached time: {non_cached_time:.2f} seconds")

print("\nOutput (non-cached):")
print(non_cached_response.content)

In [ ]:
non_cached_response.usage

## Cached Version

In [ ]:
def make_cached_api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content + "</book>",
                    "cache_control": {"type": "ephemeral"}
                },
                {
                    "type": "text",
                    "text": "What happens in chapter 5?"
                }
            ]
        }
    ]

    start_time = time.time()
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=500,
        messages=messages,
    )
    end_time = time.time()

    return response, end_time - start_time

In [ ]:
response1, duration1 = make_cached_api_call()

In [ ]:
response1

In [ ]:
response2, duration2 = make_cached_api_call()

In [ ]:
response2.usage

In [ ]:
duration2

## Prompt Caching Pricing

* Cache write tokens are 25% more expensive than base input tokens
* Cache read tokens are 90% cheaper than base input tokens
* Regular input and output tokens are priced at standard rates

## Multi-Turn Caching

In [ ]:
messages=[
    # ...long conversation so far
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hello, can you tell me more about the solar system",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    },
    {
        "role": "assistant",
        "content": "Certainly! The solar system is the collection of celestial bodies that orbit our Sun. It consists of eight planets, numerous moons, asteroids, comets, and other objects. The planets, in order from closest to farthest from the Sun, are: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Each planet has its own unique characteristics and features. Is there a specific aspect of the solar system you'd like to know more about?"
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Tell me more about Mars.",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    }
]

In [ ]:
messages=[
    # ...long conversation so far
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hello, can you tell me more about the solar system",
            }
        ]
    },
    {
        "role": "assistant",
        "content": "Certainly! The solar system is the collection of celestial bodies that orbit our Sun. It consists of eight planets, numerous moons, asteroids, comets, and other objects. The planets, in order from closest to farthest from the Sun, are: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Each planet has its own unique characteristics and features. Is there a specific aspect of the solar system you'd like to know more about?"
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Tell me more about Mars.",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    },
    {
        "role": "assistant",
        "content": "I'd love to tell you about Mars.  Mars is...."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "That's really neat.  Tell me about Pluto!",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    },
]